# 3. Frequent Patterns

This JupyterNotebook is part of an exercise series titled *Frequent Patterns*. The series itself is based on lecture *6. Mining Frequent Patterns, Associations and Correlations*. 

There are two parts:

- Part One: Implementing A Priori and FP-Growth
- Part Two: Mining Frequent Patterns in the AdventureWorks Database

Recall that we have two exercise groups. Depending on how each group progresses, some parts of these exercises may not be discussed in its entirety. If questions arise, ask them in your study group or in our StudOn forum.

## Part One: Implementing A Priori and FP-Growth

In this part we will take a closer look at the methods A Priori and FP-Growth, which are well known from the lecture. In the following, you will first implement both methods yourself step by step and then compare your implementation with the implementation of a common library.


In [ ]:
# Import the required libraries
import itertools
import pandas as pd
from dataclasses import dataclass, field

We take a look at a very small data set in this part. It was already used in the lecture and should enable you to validate your code by yourself without knowing a sample solution.

In [ ]:
# A very small data set in the form of a list (transactions) of sets (items)
dataset = [
    {"Beer", "Nuts", "Diapers"},
    {"Beer", "Coffee", "Diapers"},
    {"Beer", "Diapers", "Eggs"},
    {"Nuts", "Eggs", "Milk"},
    {"Nuts", "Coffee", "Diapers", "Eggs"},
]
dataset

### A Priori

The first method we consider is A Priori. It is a very basic approach, which requires many accesses to the data set under consideration.

#### Implementation

For our implementation, we first define a (data)class `Itemset`, which can be used to store a set of items together with the count of occurrences of these items in our data set.

In [ ]:
# The (data)class Itemset
@dataclass
class Itemset:
    # Attributes
    items: set
    occurrence_count: int = 0


# Example of usage (might be a hint for later tasks)
# Create an example Itemset
example_itemset = Itemset({"Beer", "Nuts"})

# Increase the occurrence_count
example_itemset.occurrence_count += 1

# Check whether this itemset is a subset of a bigger set of items
example_itemset.items.issubset({"Beer", "Nuts", "Diapers"})
example_itemset

We also define a class `ItemsetList`, which is a list of `Itemset`s providing some functions you might want to use in later tasks.

In [ ]:
# The class ItemsetList
class ItemsetList:
    # Constructor
    def __init__(self, itemsets: list[Itemset]):
        self.itemsets = itemsets

    # Functions
    # Return all Itemsets which are containing exactly the passed items
    def get_itemsets_with_items(self, items: set):
        return [x for x in self.itemsets if x.items == items]

    # Check if a there is at least a Itemset containing exactly the passed items
    def contains_itemset_with_items(self, items: set):
        return len(self.get_itemsets_with_items(items)) > 0

    # Return all Itemsets which are containing a superset of the passed items
    def get_itemsets_with_superset_of_items(self, items: set):
        return [x for x in self.itemsets if x.items.issuperset(items)]

    # Check if a there is at least a Itemset containing a superset of the passed items
    def contains_itemset_with_superset_of_items(self, items: set):
        return len(self.get_itemsets_with_superset_of_items(items)) > 0

    # Return all Itemsets which are containing a subset of the passed items
    def get_itemsets_with_subset_of_items(self, items: set):
        return [x for x in self.itemsets if x.items.issubset(items)]

    # Check if a there is at least a Itemset containing a subset of the passed items
    def contains_itemset_with_subset_of_items(self, items: set):
        return len(self.get_itemsets_with_subset_of_items(items)) > 0


# Example of usage (might be a hint for later tasks)
# Create an example ItemsetList
example_itemset_list = ItemsetList([])

# Add our example itemset to the list
example_itemset_list.itemsets.append(example_itemset)

# Check if there is a itemset with exactly the items Beer and Nuts
example_itemset_list.contains_itemset_with_items({"Beer", "Nuts"})

# Get the itemsets with a subset of the items Beer and Nuts
example_itemset_list.get_itemsets_with_subset_of_items({"Beer", "Nuts", "Diapers"})
example_itemset_list.itemsets

The first step in A Priori is to scan the dataset once to get all 1-itemsets. To avoid scanning the dataset multiple times during the search for frequent 1-itemsets the count of occurrences of each item is determined during that step.

<div class="alert alert-block alert-info">

**Task:** Complete the function below, which is intended to generate all 1-itemsets and their occurrence count based on a given dataset.
    
</div>

In [ ]:
# Implement a function to generate all 1-itemsets
def generate_one_itemsets(dataset):
    # Initialize an ItemsetList
    itemsets = ItemsetList([])

    # ...

    # Return the itemsets
    return itemsets


# Get all 1-itemsets (and their occurrence count) within our dataset
one_itemsets = generate_one_itemsets(dataset)
one_itemsets.itemsets

In [ ]:
# Implement a function to generate all 1-itemsets
def generate_one_itemsets(dataset):
    # Initialize an ItemsetList
    itemsets = ItemsetList([])

    # Iterate over all transactions
    for transaction in dataset:
        # Iterate over all items contained in that transaction
        for item in transaction:
            # Check whether the itemset already exists in itemsets
            if itemsets.contains_itemset_with_items({item}):
                # If yes just increment the items count
                itemsets.get_itemsets_with_items({item})[0].occurrence_count += 1
            else:
                # If no add the item to itemsets (occurrence_count has to be 1, as it is the first occurrence)
                itemsets.itemsets.append(Itemset({item}, 1))

    # Return the itemsets
    return itemsets


# Get all 1-itemsets (and their occurrence count) within our dataset
one_itemsets = generate_one_itemsets(dataset)
one_itemsets.itemsets

Only items that are occurring more often or the same number of times as defined in `minimal_support_count` are frequent 1-itemsets. For this reason, the next necessary step is to prune all itemsets that occur less frequently than this value.


<div class="alert alert-block alert-info">

**Task:** Create the function `prune_itemsets` that removes itemsets that do not satisfy `minimal_support_count` (we use a `minimal_support_count` of 3 in this example).
    
</div>

In [ ]:
# Implement a function to prune itemsets that occurred less then minimal_support times
def prune_itemsets(itemsets, minimal_support_count):
    # Initialize an ItemsetList
    frequent_itemsets = ItemsetList([])

    # ...

    # Return the itemsets
    return frequent_itemsets


# Prune every itemset occuring less then three times
frequent_one_itemsets = prune_itemsets(one_itemsets, 3)
frequent_one_itemsets.itemsets

In [ ]:
# Implement a function to prune itemsets that occurred less then minimal_support times
def prune_itemsets(itemsets, minimal_support_count):
    # Initialize an ItemsetList
    frequent_itemsets = ItemsetList([])

    # Get all itemsets that occurred at least minimal_support_count times
    # This is very similar to the functions given in ItemsetList
    # but yet it is not included in ItemsetList to provide a little challenge
    frequent_itemsets.itemsets = [
        x for x in itemsets.itemsets if x.occurrence_count >= minimal_support_count
    ]

    # Return the itemsets
    return frequent_itemsets


# Prune every itemset occuring less then three times
frequent_one_itemsets = prune_itemsets(one_itemsets, 3)
frequent_one_itemsets.itemsets

One of the most important principles that A Priori uses is that only itemsets that are themselves frequent can lead to supersets that are frequent. So to find possible candidates for frequent 2-itemsets, only the found frequent 1-itemsets have to be combined to 2-itemsets.

<div class="alert alert-block alert-info">
    
**Task:** Implement `generate_candidates` so that it can be used to generate length-(k+1) candidate itemsets from lenght-k frequent itemsets. You are allowed to use `itertools`.
    
</div>

In [ ]:
# Implement a function to generate length-k+1 candidate itemsets from length-k frequent itemsets
def generate_candidates(frequent_k_itemsets):
    # Initialize an ItemsetList
    candidates = ItemsetList([])

    # ...

    # Return the candidates
    return candidates


# Generate the candidates of the second level
two_candidates = generate_candidates(frequent_one_itemsets)
two_candidates.itemsets

In [ ]:
# Implement a function to generate length-k+1 candidate itemsets from length-k frequent itemsets
def generate_candidates(frequent_k_itemsets):
    # Initialize an ItemsetList
    candidates = ItemsetList([])

    # Get k
    k = len(frequent_k_itemsets.itemsets[0].items)

    # Iterate over the frequent_k_itemsets to get all items contained in at least a single frequent_k_itemset
    items = set()
    for itemset in frequent_k_itemsets.itemsets:
        # Add the items of the itemset to items
        items = items.union(itemset.items)

    # Find all combinations with lenght k+1
    for combination in itertools.combinations(items, k + 1):
        # Check that all subsets with length k are part of frequent_k_itemsets
        all_k_subsets_are_part_of_frequent_k_itemsets = True

        for i in range(k + 1):
            # Convert combination into a list
            # (== copy of the combination)
            k_subset = list(combination)

            # Remove the i-th element
            k_subset.pop(i)

            # Convert the list into set
            k_subset = set(k_subset)

            # Check if k_subset is contained in frequent_k_itemsets
            if not frequent_k_itemsets.contains_itemset_with_items(k_subset):
                # A k_subset is not part of frequent_k_itemsets
                # => The combination is no candidate for k+1
                all_k_subsets_are_part_of_frequent_k_itemsets = False

                # Of course we can skipping further checking now
                break

        # If all are part of frequent_k_itemsets the combination is a candidate
        if all_k_subsets_are_part_of_frequent_k_itemsets:
            candidates.itemsets.append(Itemset(set(combination), 0))

    # Return the candidates
    return candidates


# Generate the candidates of the second level
two_candidates = generate_candidates(frequent_one_itemsets)
two_candidates.itemsets

In [ ]:
# You might want to check the list with an other example as well
extra_frequent_two_itemsets = ItemsetList(
    [
        Itemset({"Football", "Shoes"}),
        Itemset({"Football", "Glasses"}),
        Itemset({"Shoes", "Glasses"}),
        Itemset({"Glasses", "Tissues"}),
    ]
)

# Generate the candidates of the third level
extra_three_candidates = generate_candidates(extra_frequent_two_itemsets)
extra_three_candidates.itemsets

After generating candidates, the next step is to scan the dataset to find out how often which candidate occurs.

<div class="alert alert-block alert-info">
    
**Task:** Finalize the `scan_candidates` function, which is used to determine how often each candidate Itemset occurs in the dataset.
    
</div>

In [ ]:
# Implement a function to determine how often each candidate Itemset occurs in the dataset
def scan_candidates(dataset, candidates):

    # ...

    # Return the candidates
    return candidates


# Determine how often each candidate Itemset occurs
two_candidates = scan_candidates(dataset, two_candidates)
two_candidates.itemsets

In [ ]:
# Implement a function to determine how often each candidate Itemset occurs in the dataset
def scan_candidates(dataset, candidates):
    # Iterate over all transactions
    for transaction in dataset:
        # Get all candidates itemsets that are a subset of the items occurring in the dataset
        subset_candidates = candidates.get_itemsets_with_subset_of_items(transaction)

        # Increase the occurrence count of each candidate Itemset being a subset of the items occurring in the dataset
        for subset_candidate in subset_candidates:
            subset_candidate.occurrence_count += 1

    # Return the candidates
    return candidates


# Determine how often each candidate Itemset occurs
two_candidates = scan_candidates(dataset, two_candidates)
two_candidates.itemsets

Once the number of occurrences of each candidate has been determined, `prune_itemsets` can be used again to remove the candidates that do not match the `minimal_support_count` of 3. 

In [ ]:
# Prune all Itemset below the minimal_support_count of 3
frequent_two_itemsets = prune_itemsets(two_candidates, 3)
frequent_two_itemsets.itemsets

After the initial execution of `generate_one_itemsets`, the functions `prune_itemsets`, `generate_candidates` and `scan_candidates` are executed in a loop until no further candidates or frequent itemsets can be found.

<div class="alert alert-block alert-info">
    
**Task:** Write the function a_priori that uses the functions `generate_one_itemsets`, `prune_itemsets`, `generate_candidates` and `scan_candidates` to perform a complete run of A Priori for an arbitrarily large data set.
    
</div>

In [ ]:
# Implement an A Priori wrapper
def a_priori(dataset, minimal_support_count):
    # Initialize an ItemsetList
    frequent_itemsets = ItemsetList([])

    # ...

    # Return the frequent_itemsets
    return frequent_itemsets


# Get all frequent itemsets within our dataset satisfing the minimal_support_count of 3
frequent_itemsets = a_priori(dataset, 3)
frequent_itemsets.itemsets

In [ ]:
# Implement an A Priori wrapper
def a_priori(dataset, minimal_support_count):
    # Initialize an ItemsetList
    frequent_itemsets = ItemsetList([])

    # Start by generating all 1-itemsets and make the first candidates for becoming frequent itemsets
    candidate_itemsets = generate_one_itemsets(dataset)

    # Start the loop that will run as long as there are candidate_itemsets
    while len(candidate_itemsets.itemsets) > 0:
        # Prune the candidate itemsets not satisfing the minimal_support_count
        frequent_k_itemsets = prune_itemsets(candidate_itemsets, minimal_support_count)

        # If there are no frequent_k_itemset we might also break the loop (second termination criterion)
        if len(frequent_k_itemsets.itemsets) == 0:
            break

        # Otherwise we should add the found frequent k-itemsets to the main list of frequent_itemsets
        frequent_itemsets.itemsets.extend(frequent_k_itemsets.itemsets)

        # Prepare the next loop run
        # Generate possible candidates
        candidate_itemsets = generate_candidates(frequent_k_itemsets)

        # Determine how often each candidate occurs
        candidate_itemsets = scan_candidates(dataset, candidate_itemsets)

    # Return the frequent_itemsets
    return frequent_itemsets


# Get all frequent itemsets within our dataset satisfing the minimal_support_count of 3
frequent_itemsets = a_priori(dataset, 3)
frequent_itemsets.itemsets

#### Libary: Mlxtend

Of course, it's tedious to program A Priori yourself every time you need it. For this reason, there are already some libraries that contain appropriate methods. On this worksheet we use `mlxtend`.

In [ ]:
# Import the required packages of mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

To be able to use the function `apriori` from `mlxtend` to get the frequent itemsets contained in our dataset, we first have to transform it into a suitable format.

<div class="alert alert-block alert-info">
    
**Task:** Take a look at the `mlxtend` [documentation](http://rasbt.github.io/mlxtend/USER_GUIDE_INDEX/) for information on how dataset must be structured for `apriori` and preprocess our `dataset` accordingly.
    
</div>

In [ ]:
# Preprocess the dataset

In [ ]:
# Preprocess the dataset
# Create a TransactionEncoder
transaction_encoder = TransactionEncoder()

# Use the TransactionEncoder to transform the dataset into a one-hot encoded NumPy boolean array
one_hot_encoded_dataset = transaction_encoder.fit(dataset).transform(dataset)

# Transform the one-hot encoded array into a pandas DataFrame
preprocessed_dataset = pd.DataFrame(
    one_hot_encoded_dataset, columns=transaction_encoder.columns_
)
preprocessed_dataset

After this preparation, the determination of the frequent itemset in our dataset is possible by using `apriori`. 

<div class="alert alert-block alert-info">
    
**Task:** Using `apriori` from `mlxtend`, determine the frequent itemsets in our dataset. Use a `min_support` comparable to the value we used in the previous section (`minimal_support_count` of 3).
    
</div>

In [ ]:
# Use apriori from mlxtend to determine the frequent itemsets in our dataset

In [ ]:
# Use apriori from mlxtend to determine the frequent itemsets in our dataset
# Min support has to be 0.6 as there are 5 tuples in our dataset
# => min_support of 0.6 == minimal_suport_count of 3 for 5 tuples)
apriori(preprocessed_dataset, min_support=0.6, use_colnames=True)

There are several differences between your own implementation and mlxtend's. 

<div class="alert alert-block alert-info">
    
**Task:** Consider what differences there are between your implementation and `mlxtend`'s implementation of `apriori` for the user of these functions. 
    
</div>

Write down your solution here:

Of course, the individual details also depend on your specific implementation. However, based on our specifications, it is to be expected that at least the following things will differ:

- <b>The format of the input:</b><br /> 
Both `mlxtend`s implementation, and your own implementation require a specific format of the dataset on entry. 

- <b>The format of the output:</b><br /> 
The format of the output is also different for both variants. 

### FP-Growth

Much more complex than A Priori is the FP-Growth approach. However, since this approach requires only two scans of the data set, it also has a clear advantage, especially for larger projects.

#### Implementation

While we can continue to use some of the classes already introduced for A Priori in our implementation, FP-Growth requires some additional structures.

The first thing that comes to mind is the FP-Tree, which consists of the root node and nodes for individual items. In the following we provide you with the classes `FPTree`, `RootNode` and `ItemNode`, that are supposed to serve as the correspondig data structures. Similar to the provided class `ItemsetList` they also provide some basic functionality you are allowed to use in your FP-Growth implementation.

<div class="alert alert-block alert-warning">

**Note**: Don't be put off by the size of the classes. In order to simplify the FP-Growth implementation for you, some functionalities have been outsourced to these classes. 
    
First get a rough overview of the respective class (variable and function names) and come back to the details when you need the functionality.
    
</div>

In [ ]:
# The class FPTree
class FPTree:
    # Constructor
    def __init__(self):
        # Create a root node for the tree
        self.root = RootNode()

    # Add a set of items to the tree
    # This function can be used for adding transactions to the FPTree (occurrence_count of a single transaction = 1)
    # Or for adding a single frequent pattern that is part of a conditional frequent pattern base to the conditional FPTree (occurrence_count is the occurrence count of this single frequent pattern)
    def add_items_to_tree(
        self, items, f_list: list[Itemset], occurrence_count: int = 1
    ):
        # Set the root node as current node
        current_node = self.root

        # Interate through the f_list
        for item in f_list.itemsets:
            # Check if the item is part items
            if item.items.issubset(items):
                # Check if the item is already present as a child of the current node
                if len([x for x in current_node.childs if x.item in item.items]) > 0:
                    # Set the node with the item to be the current_node
                    current_node = [
                        x for x in current_node.childs if x.item in item.items
                    ][0]

                    # Increase the occurence count of that node
                    current_node.occurrence_count += occurrence_count
                else:
                    # Create a new node
                    new_node = ItemNode(
                        list(item.items)[0], occurrence_count, current_node
                    )

                    # Set the new_node as current_node
                    current_node = new_node

    # Construct the HeaderTable of the Tree
    def get_header_table(self):
        # Create a HeaderTable
        header_table = HeaderTable()

        # Call the add_to_header_table() function of the RootNode
        self.root.add_to_header_table(header_table)

        # Return the header_table
        return header_table

    # Get all item nodes within the FP-tree
    def get_all_item_nodes(self):
        # Ask the root node
        return self.root.get_all_item_nodes()

    # Check whether the FPTree has only a single path
    def is_single_path(self):
        # Ask the root node
        return self.root.is_single_path()

    # Check whether the FPTree is empty
    def is_empty(self):
        # Ask the root node
        return self.root.is_empty()

    # Print the tree
    def print_tree(self):
        # Print the subtree starting with the root node
        self.root.print_subtree()

In [ ]:
# The class RootNode
class RootNode:
    # Constructor
    def __init__(self):
        # Set some member variables
        self.item = "Root-Node"
        self.childs = list()

    # Add to HeaderTable
    def add_to_header_table(self, header_table):
        # The root node itself is not part of the HeaderTable

        # But the all childs have to be added
        for child in self.childs:
            # Recursive call of the add_to_header_table() function
            child.add_to_header_table(header_table)

    # Get the predecessors (there is no predecessor for the RootNode)
    def get_predecessors(self):
        # Return an empty list as there are no predecessors
        return []

    # Get all item nodes within the FP-tree
    def get_all_item_nodes(self):
        # Init an empty list to add all ItemNodes to
        node_list = []

        # Go through all childs and add there lists to this node_list
        for child in self.childs:
            node_list.extend(child.get_all_item_nodes())

        return node_list

    # Check whether the RootNode has only a single path behind it
    def is_single_path(self):
        # If there is more then one child return False
        if len(self.childs) > 1:
            return False
        # If there is exactly one child ask that child if there is only a single path
        elif len(self.childs) == 1:
            return self.childs[0].is_single_path()
        # If there are no childs there is only a single path
        else:
            return True

    # Check whether the FPTree is empty
    def is_empty(self):
        # If there at least one child return False
        if len(self.childs) >= 1:
            return False
        # Otherwise return true
        return True

    # Print the subtree
    def print_subtree(self):
        # Print the root itself
        print(self.item)

        # Print the childs
        for child in self.childs:
            child.print_subtree(1)

In [ ]:
# The class ItemNode
class ItemNode:
    # Constructor
    def __init__(self, item: str, occurrence_count: int, parent):
        # Save the arguments
        self.item = item
        self.occurrence_count = occurrence_count
        self.parent = parent

        # Set the other parameters used later in the lifespan
        self.childs = list()

        # Save the node as child in the parent node
        parent.childs.append(self)

    # Add to HeaderTable
    def add_to_header_table(self, header_table):
        # Check if there already is a element for this item
        if len([x for x in header_table.elements if x.item == self.item]) > 0:
            # If there is already an element for this item in the HeaderTable
            # Get the element
            header_table_element = [
                x for x in header_table.elements if x.item == self.item
            ][0]

            # Add the occurence count to the overall occurrence count
            header_table_element.overall_occurrence_count += self.occurrence_count

            # Add the ItemNode itself to the element_node_links
            header_table_element.node_links.append(self)
        else:
            # If there is no element for this item in the HeaderTable
            # Create a new HeaderTableElement
            header_table_element = HeaderTableElement(
                self.item, self.occurrence_count, [self]
            )

            # Add it to the HeaderTable
            header_table.elements.append(header_table_element)

        # Do a recursive call to add_to_header_table for all childs
        for child in self.childs:
            # Recursive call of the add_to_header_table() function
            child.add_to_header_table(header_table)

    # Get the predecessors of this element node (excluding the RootNode)
    def get_predecessors(self):
        # Get the parents predecessors
        predecessors = self.parent.get_predecessors()

        # Add the parent to the predecessors if it is not the RootNode
        if type(self.parent) != RootNode:
            predecessors.append(self.parent)

        # Return the predecessors
        return predecessors

    # Get all item nodes within the FP-tree
    def get_all_item_nodes(self):
        # Init an list and add this node to it
        node_list = [self]

        # Go through all childs and add their lists to this node_list
        for child in self.childs:
            node_list.extend(child.get_all_item_nodes())

        return node_list

    # Check whether the RootNode has only a single path behind it
    def is_single_path(self):
        # If there is more then one child return False
        if len(self.childs) > 1:
            return False
        # If there is exactly one child ask that child if there is only a single path
        elif len(self.childs) == 1:
            return self.childs[0].is_single_path()
        # If there are no childs there is only a single path
        else:
            return True

    # Print the subtree
    def print_subtree(self, level):
        # Print the node itself
        print(
            (" " * (level - 1) * 2)
            + "├── "
            + self.item
            + ": "
            + str(self.occurrence_count)
        )

        # Print the childs
        for child in self.childs:
            child.print_subtree(level + 1)

In addition to the central element FP-Tree, FP-Growth also uses another smaller data structure, the header table. This data structure is provided by us, too. The class `HeaderTable` represents the table, which consists of several elements, the `HeaderTableElement`s.

In [ ]:
# The class HeaderTable
class HeaderTable:
    # Constructor
    def __init__(self):
        # Save the arguments
        self.elements = list()

    # Print the table
    def print_table(self):
        # For all elements of the table
        for element in self.elements:
            # Print the element
            element.print_element()

In [ ]:
# The class HeaderTableElement
class HeaderTableElement:
    # Constructor
    def __init__(self, item: str, overall_occurrence_count: int, node_links: list):
        # Save the arguments
        self.item = item
        self.overall_occurrence_count = overall_occurrence_count
        self.node_links = node_links  # Links to every item node regarding our item

    # Print the element
    def print_element(self):
        print(
            self.item
            + ": "
            + str(self.overall_occurrence_count)
            + " - Count of linked nodes: "
            + str(len(self.node_links))
        )

A first important step in FP-Growth can be done with the help of the functions already implemented for A Priori. Namely, the frequent 1-itemsets of a dataset have to be determined.

<div class="alert alert-block alert-info">
    
**Task:** Determine the frequent 1-itemsets occuring at least two times for our dataset `dataset` and store them as `frequent_one_itemsets`. You may use functions from your A Priori implementation to do this.
    
</div>

In [ ]:
# Find the frequent 1-itemsets occurring at least two times

In [ ]:
# Find the frequent 1-itemsets occurring at least two times
one_itemsets = generate_one_itemsets(dataset)
frequent_one_itemsets = prune_itemsets(one_itemsets, 2)
frequent_one_itemsets.itemsets

Using the frequent 1-itemsets, the next step in the FP-Growth algorithm is to create the so-called f-list: An list of the frequent 1-itemsets in frequency-descending order.

<div class="alert alert-block alert-info">
    
**Task:** Complete `create_f_list`, which computes the f-list required for FP-Growth from frequent 1-itemsets. Make sure that the passed frequent 1-itemsets are not overwritten by the function.
    
</div>

In [ ]:
# Complete the function create_f_list

In [ ]:
# Complete the function create_f_list
def create_f_list(frequent_one_itemsets: ItemsetList):
    # Copy the itemset list
    f_list = ItemsetList(frequent_one_itemsets.itemsets.copy())

    # Sort the list copy and return it
    f_list.itemsets.sort(key=lambda x: x.occurrence_count, reverse=True)

    # Return the list
    return f_list


# Create the f-list
f_list = create_f_list(frequent_one_itemsets)
f_list.itemsets

Based on the f-list, the initial FP tree can now be created. For this purpose, the items occurring in each transaction are added to a hierarchical tree structure according to their order in the f-list. 

<div class="alert alert-block alert-info">
    
**Task:** Take a look at `add_items_to_tree` within the `FPTree` class and consider how it can be used to create the initial FP-Tree from the dataset and f-list. Then complete the function below accordingly.
    
</div>

In [ ]:
# Complete the function construct_fp_tree used to construct the inital (non-conditional) FP-Tree
def construct_fp_tree(dataset, f_list):
    # Initialize an FPTree
    fp_tree = FPTree()

    # ...

    # Return the FPTree
    return fp_tree


# Construct the FP-Tree
fp_tree = construct_fp_tree(dataset, frequent_one_itemsets)

# Print the FP-Tree
print("FP-Tree:")
fp_tree.print_tree()

# Display the header table
print("\nHeader table:")
fp_tree.get_header_table().print_table()

In [ ]:
# Complete the function construct_fp_tree used to construct the inital (non-conditional) FP-Tree
def construct_fp_tree(dataset, f_list):
    # Initialize an FPTree
    fp_tree = FPTree()

    # Iterate over all transactions
    for transaction in dataset:
        # Add the transaction to the FPTree
        fp_tree.add_items_to_tree(transaction, f_list, 1)

    # Return the FPTree
    return fp_tree


# Construct the FP-Tree
fp_tree = construct_fp_tree(dataset, f_list)

# Print the FP-Tree
print("FP-Tree:")
fp_tree.print_tree()

# Display the header table
print("\nHeader table:")
fp_tree.get_header_table().print_table()

The FP tree can be used to determine the so-called conditional pattern base for any contained frequent 1-itemsets. This conditional pattern base describes which other items occur before the respective one in the FP tree and how often they can be found with the item in the data set.

<div class="alert alert-block alert-info">
    
**Task:** Extend `get_conditional_pattern_base` so that it can be used to determine the conditional pattern base for any item within a FP-Tree.
    
</div>

In [ ]:
# Extend get_conditional_pattern_base to be able to determine a conditional pattern base for any itemset
def get_conditional_pattern_base(item, fp_tree):
    # Initialize an ItemsetList for the conditional pattern base
    conditional_pattern_base = ItemsetList([])

    # ...

    # Return the found conditional_pattern_base
    return conditional_pattern_base


# Find the conditional pattern base of "Eggs"
conditional_pattern_base = get_conditional_pattern_base("Eggs", fp_tree)
conditional_pattern_base.itemsets

In [ ]:
# Extend get_conditional_pattern_base to be able to determine a conditional pattern base for any itemset
def get_conditional_pattern_base(itemset, fp_tree):
    # Initialize an ItemsetList for the conditional pattern base
    conditional_pattern_base = ItemsetList([])

    # Get the header table of the fp_tree
    header_table = fp_tree.get_header_table()

    # Search for the items element within the header_table
    header_table_element = [x for x in header_table.elements if x.item == itemset]

    # If there is an element (otherwise there will be no conditional_pattern_base)
    if len(header_table_element) == 1:
        # In this case we can switch out the list of elements for a single element
        header_table_element = header_table_element[0]

        # For every linked node (ItemNodes regarding our item)
        for linked_node in header_table_element.node_links:
            # Get the predecessors
            predecessors = linked_node.get_predecessors()

            # If there are predecessors
            if len(predecessors) > 0:
                # Get all the the items part of the predecessors
                predecessor_items = {x.item for x in predecessors}

                # Create an Itemset with the occurrence_count set to the occurrence_count of the linked_node
                # and the items set to the predecessor_items
                itemset = Itemset(predecessor_items, linked_node.occurrence_count)

                # Add the items to the conditional_pattern_bases
                conditional_pattern_base.itemsets.append(itemset)

    # Return the found conditional_pattern_base
    return conditional_pattern_base


# Find the conditional pattern base of "Eggs"
conditional_pattern_base = get_conditional_pattern_base("Eggs", fp_tree)
conditional_pattern_base.itemsets

The conditional pattern base can be used to create a new FP tree. This conditional FP tree shows what the initial FP tree would look like if the selected itemset absolutely had to occur.

<div class="alert alert-block alert-info">
    
**Task:** Implement the funtionality of `construct_conditional_fp_tree` so that it can be used to create a conditional FP-Tree out of any conditional pattern base. Remember what you have learned about the `FPTree`s `add_items_to_tree`.
    
</div>

In [ ]:
# Implement a function to construct a conditional FP-tree out of a conditional pattern base
def construct_conditional_fp_tree(conditional_pattern_base, f_list):
    # Initialize an FPTree
    fp_tree = FPTree()

    # ...

    # Return the FPTree
    return fp_tree


# Construct the conditional FP-tree for the conditional pattern base of "Eggs"
conditional_fp_tree = construct_conditional_fp_tree(
    conditional_pattern_base, frequent_one_itemsets
)
conditional_fp_tree.print_tree()

In [ ]:
# Implement a function to construct a conditional FP-tree out of a conditional pattern base
def construct_conditional_fp_tree(conditional_pattern_base, f_list):
    # Initialize an FPTree
    fp_tree = FPTree()

    # Iterate over all conditional patterns
    for conditional_pattern in conditional_pattern_base.itemsets:
        # Add the conditional_pattern to the FPTree
        fp_tree.add_items_to_tree(
            conditional_pattern.items, f_list, conditional_pattern.occurrence_count
        )

    # Return the FPTree
    return fp_tree


# Construct the conditional FP-tree for the conditional pattern base of "Eggs"
conditional_fp_tree = construct_conditional_fp_tree(conditional_pattern_base, f_list)
conditional_fp_tree.print_tree()

As introduced in the lecture, FP-Growth is ultimately a recursive process in which conditional pattern bases for all elements of the header table are generated from an FP-Tree, from which conditional fp trees are then generated, after which a new stage of recursion is performed.

In general, the recursion within FP-Growth can be represented as pseudocode as follows:

```
FP-GROWTH-RECURSION-STEP(FP-TREE, BASE-PATTERN, F-LIST):
    Initialize an empty ITEMSETS-FOUND variable

    If(BASE-PATTERN is not null):
        Add the BASE-PATTERN to the ITEMSETS-FOUND

    If(FP-TREE has only one path):
        For each possible COMBINATION of the NODES within the FP-TREE:
            Combine the items of the BASE-PATTERN with the items of the COMBINATION to build a new ITEMSET
            Set the support of the ITEMSET to the support of the rarest item in the COMBINATION
            Add the ITEMSET to the ITEMSETS-FOUND
    Else:
        For each ITEM in the HEADER-TABLE of the FP-TREE:
            If(BASE-PATTERN is null):
                Set the ITEM to be the NEW-BASE-PATTERN 
                Set the support of the NEW-BASE-PATTERN to the ITEMs overall occurrence count within the HEADER-TABLE
            Else:
                Combine the items of the BASE-PATTERN with the ITEM to become the NEW-BASE-PATTERN 
                Set the support of the NEW-BASE-PATTERN to MIN(BASE-PATTERNs support, ITEMs overall occurrence count)
             
            Determine the CONDITIONAL-PATTERN-BASE of the NEW-BASE-PATTERN
            Use the CONDITIONAL-PATTERN-BASE to build the CONDITIONAL-FP-TREE
            Call FP-GROWTH-RECURSION-STEP recursively with CONDITIONAL-FP-TREE and NEW-BASE-PATTERN
            Add the result of the recursive call to the ITEMSETS-FOUND
            
    Return the ITEMSETS-FOUND
```

<div class="alert alert-block alert-info">
    
**Task:** Based on this pseudo_code, complete the function `fp_growth_recursion_step` below.
    
</div>

In [ ]:
# Complete the function fp_growth_recursion_step to implement the FP-Growth recursion step
def fp_growth_recursion_step(fp_tree, base_pattern, f_list):
    # Initalize an ItemsetList to return all the patterns found during this fp_growth step
    patterns = ItemsetList([])

    # ...

    # Return the found patterns
    return patterns


# Use the method to get all Itemsets based on the base tree
fp_growth_recursion_step(fp_tree, None, frequent_one_itemsets).itemsets

In [ ]:
# Complete the function fp_growth_recursion_step to implement the FP-Growth recursion step
def fp_growth_recursion_step(fp_tree, base_pattern, f_list):
    # Initalize an ItemsetList to return all the patterns found during this fp_growth step
    patterns = ItemsetList([])

    # If the base path is not none add it to the pattern list
    if base_pattern != None:
        patterns.itemsets.append(base_pattern)

    # If the fp_tree has only one path we can directly generate every pattern by combining each item with each item
    if fp_tree.is_single_path():

        # Get all nodes within the single path
        node_list = fp_tree.get_all_item_nodes()

        # There are combinations between length 1 - (incl.) length len(node_list)
        for length in range(1, len(node_list) + 1):
            # Create every possible combination with this length
            for combination in itertools.combinations(node_list, length):
                # Create a new Itemset for this pattern by merging the base pattern with the newly found combination
                # The occurrence count is set to the lowest occurrence count in that combination
                pattern = Itemset(
                    {x.item for x in combination}.union(base_pattern.items),
                    min([x.occurrence_count for x in combination]),
                )

                # Add the pattern to the patterns
                patterns.itemsets.append(pattern)
    # Otherwise we have to perform a recursive search
    else:
        # We have to generate the conditional pattern base for every item in the header_table
        for header_table_element in fp_tree.get_header_table().elements:
            if base_pattern == None:
                # Set the new_base_pattern to be the header_table_item
                new_base_pattern = Itemset(
                    {header_table_element.item},
                    header_table_element.overall_occurrence_count,
                )
            else:
                # Merge the new item into the base_pattern to become the new_base_pattern
                new_base_pattern = Itemset(
                    {header_table_element.item}.union(base_pattern.items),
                    min(
                        base_pattern.occurrence_count,
                        header_table_element.overall_occurrence_count,
                    ),
                )

            # Construct the conditional patterns base
            new_conditional_pattern_base = get_conditional_pattern_base(
                header_table_element.item, fp_tree
            )
            new_conditional_fp_tree = construct_conditional_fp_tree(
                new_conditional_pattern_base, f_list
            )

            # Start the recursion
            patterns.itemsets.extend(
                fp_growth_recursion_step(
                    new_conditional_fp_tree, new_base_pattern, f_list
                ).itemsets
            )

    # Return the found patterns
    return patterns


# Use the method to get all Itemsets based on the base tree
fp_growth_recursion_step(fp_tree, None, f_list).itemsets

The last thing that needs to be done with FP-Growth is to merge the individual components into a wrapper function. 

<div class="alert alert-block alert-info">
    
**Task:** Write a function `fp_growth` that, for a given dataset and a given minimal support count, performs all steps of the FP-Growth algorithm up to determining all frequent itemsets. Take into account that after execution of `fp_growth_recursion_step` itemsets that do not meet the minimum support count must be pruned.
    
</div>

In [ ]:
# Complete the function fp_growth combining all the functions to get a full FP-Growth implementation
def fp_growth(dataset, minimal_support_count):

    # ...
    return ItemsetList([])


# Get all frequent patterns with a minimal support count 2 or higher
fp_growth(dataset, 2).itemsets

In [ ]:
# Complete the function fp_growth combining all the functions to get a full FP-Growth implementation
def fp_growth(dataset, minimal_support_count):
    # Find the frequent 1-itemsets
    one_itemsets = generate_one_itemsets(dataset)
    frequent_one_itemsets = prune_itemsets(one_itemsets, minimal_support_count)

    # Create the f-list
    frequent_one_itemsets.itemsets.sort(key=lambda x: x.occurrence_count, reverse=True)
    f_list = frequent_one_itemsets

    # Construct the initital FP-Tree
    fp_tree = construct_fp_tree(dataset, f_list)

    # Use the recursive methode to get all n-itemsets (n > 1)
    n_itemsets = fp_growth_recursion_step(fp_tree, None, frequent_one_itemsets)

    # Prune all non frequent n-itemsets
    frequent_n_itemsets = prune_itemsets(n_itemsets, minimal_support_count)

    # Return the frequent_n_itemsets
    return frequent_n_itemsets


# Get all frequent patterns with a minimal support count 2 or higher
fp_growth(dataset, 2).itemsets

#### Libary: Mlxtend

Just with like A Priori, there is also a corresponding function for FP-Growth included in `mlxtend`.

In [ ]:
# Import the required packages of mlxtend
from mlxtend.frequent_patterns import fpgrowth

Again `mlxtend` expects a certain input format.

<div class="alert alert-block alert-info">
    
**Task:** Take a look at the `mlxtend` [documentation](http://rasbt.github.io/mlxtend/USER_GUIDE_INDEX/) for information on how dataset must be structured for `fpgrowth` and preprocess our `dataset` accordingly.
    
</div>

In [ ]:
# Preprocess the dataset

In [ ]:
# Preprocess the dataset
# Create a TransactionEncoder
transaction_encoder = TransactionEncoder()

# Use the TransactionEncoder to transform the dataset into a one-hot encoded NumPy boolean array
one_hot_encoded_dataset = transaction_encoder.fit(dataset).transform(dataset)

# Transform the one-hot encoded array into a pandas DataFrame
preprocessed_dataset = pd.DataFrame(
    one_hot_encoded_dataset, columns=transaction_encoder.columns_
)
preprocessed_dataset

After this preparation, the determination of the frequent itemset in our dataset is possible by using `fpgrowth`. 

<div class="alert alert-block alert-info">
    
**Task:** Using `fpgrowth` from `mlxtend`, determine the frequent itemsets in our dataset. Use a `min_support` comparable to the value we used in the previous section (`minimal_support_count` of 2).
    
</div>

In [ ]:
# Use fpgrowth from mlxtend to determine the frequent itemsets in our dataset

In [ ]:
# Use fpgrowth from mlxtend to determine the frequent itemsets in our dataset
fpgrowth(preprocessed_dataset, min_support=0.4, use_colnames=True)

There are several differences between your own implementation and mlxtend's. 

<div class="alert alert-block alert-info">
    
**Task:** Consider what differences there are between your implementation and `mlxtend`'s implementation of `fpgrowth` for the user of these functions. 
    
</div>

Write down your solution here:

Of course, the individual details also depend on your specific implementation. However, based on our specifications, it is to be expected that at least the following things will differ:

- <b>The format of the input:</b><br /> 
Both `mlxtend`s implementation, and your own implementation require a specific format of the dataset on entry. 

- <b>The format of the output:</b><br /> 
The format of the output is also different for both variants. 

## Part Two: Mining Frequent Patterns in the AdventureWorks Database

Whereas in Part One you worked on a very small and therefore non-realistic data set, you will now apply your knowledge of Frequent Patterns to a more realistic scenario. Imagine this:

*You are an employee in the fictitious company Adventure Works GmbH. Your job is to find out which of the company's products are frequently bought together. To start with, the management wants you to find the ten most "relevant" product pairs bought together.*

*You get access to the OLTP database of the company. You are also told by colleagues who are experts in the database that information about individual transactions can be found in the relation `TransactionHistory`. The resolution of ProductIDs into real product names can be done with the help of the relation `Product`.*

*From other similar projects of the company you also already know the required libraries and the code to connect to the OLTP database:*

In [ ]:
# Import required libraries
import os
import tempfile
import sqlite3
import urllib.request
import pandas as pd

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

In [ ]:
# Create a temporary directory
dataset_folder = tempfile.mkdtemp()

# Build path to database
database_path = os.path.join(dataset_folder, "adventure-works.db")

# Get the database
urllib.request.urlretrieve(
    "https://github.com/FAU-CS6/KDD-Databases/raw/main/AdventureWorks/adventure-works.db",
    database_path,
)

# Open connection to the adventure-works.db
connection = sqlite3.connect(database_path)

### Finding the Ten Most "Relevant" Frequent Patterns

Within this worksheet, you are now given two options: You can first tackle the task independently as you would actually have to in this scenario or you can choose the guided path, in which we accompany you step by step from loading the DataFrames to converting the Frequent Itemsets into Association Rules by spliting up the big problem into small tasks.

We recommend that you first try out the "independent" version and only switch to the guided version if you encounter problems.


<div class="alert alert-block alert-warning">
    
**Note:** In both cases there is an other section at the end of this worksheet. Do not skip it, regardless of your decission in this section. 
    
</div>

#### Option 1: Solve the Assignment Independently

In this variant we don't give you anything except for the description of the scenario, the libraries to use, the database connection, and some code cells (just add more if needed, since we only added more than one in the first place to get the tasks of Option 2 out of your view). 

However, you do get one extra small tip: If you have successfully determined the frequent itemsets, you may take another look at the list of libraries. There you will probably find a function to determine the association rules from the frequent itemsets. 

<div class="alert alert-block alert-info">
    
**Task:** Find the ten most "relevant" frequent patterns in the OLTP database of the fictitious Adventure Works GmbH. You have to decide every step from loading the DataFrames to determining the Association Rules from the Frequent Itemsets.
    
</div>

In [ ]:
# Find the ten most "relevant" frequent patterns (Code placeholder 01/10)

In [ ]:
# Find the ten most "relevant" frequent patterns (Code placeholder 02/10)

In [ ]:
# Find the ten most "relevant" frequent patterns (Code placeholder 03/10)

In [ ]:
# Find the ten most "relevant" frequent patterns (Code placeholder 04/10)

In [ ]:
# Find the ten most "relevant" frequent patterns (Code placeholder 05/10)

In [ ]:
# Find the ten most "relevant" frequent patterns (Code placeholder 06/10)

In [ ]:
# Find the ten most "relevant" frequent patterns (Code placeholder 07/10)

In [ ]:
# Find the ten most "relevant" frequent patterns (Code placeholder 08/10)

In [ ]:
# Find the ten most "relevant" frequent patterns (Code placeholder 09/10)

In [ ]:
# Find the ten most "relevant" frequent patterns (Code placeholder 10/10)

In [ ]:
# Sample solution => See Option 2

#### Option 2: Solve the Assignment by Solving Small Tasks 

Any large assignment can, of course, be broken down into many smaller steps. For a KDD task, where only a database and some relevant relations are given, the first important step is to first get familiar with the given data. 


##### Getting to Know `TransactionHistory` and `Product`

To become familiar with the data within 'TransactionHistory' and 'Product', records must first be loaded. Since we don't know anything about the relations yet, it would make sense to load all attributes for both datasets first and also not to perform a selection of individual tuples.

<div class="alert alert-block alert-info">
    
**Task:** Load the relations `TransactionHistory` and `Product` into two individual DataFrames and display the first ten rows of each DataFrame. (Hint: You might want to look at exercise sheet 2 (a-c) to get to know methods of loading relations into a DataFrame)
    
</div>

In [ ]:
# Load TransactionHistory into a DataFrame and display the first ten rows

In [ ]:
# Load Product into a DataFrame and display the first ten rows

In [ ]:
# Load TransactionHistory into a DataFrame and display the first ten rows
transaction_history_df = pd.read_sql_query(
    "SELECT * FROM TransactionHistory", connection
)
transaction_history_df.head(10)

In [ ]:
# Load Product into a DataFrame and display the first ten rows
product_df = pd.read_sql_query("SELECT * FROM Product", connection)
product_df.head(10)

As our fictitious colleagues told us in the scenario, the `TransactionHistory` seems to contain information about individual transactions. However, it is not yet possible to see how it is possible to determine which products (probably identified via the `ProductID`) are purchased together. 

With the Product table, the only important information seems to what `ProductID` leads to which `Name`, which is why we can get straight to the problem in the `TransactionHistory`:

We might therefore assume that we are looking for products that are purchased in the same transaction. Since the attribute `TransactionID` probably uniquely identifies each transaction, it would make sense to test this hypothesis by determining whether there are `TransactionID`s with more than one linked `ProductID`. 

<div class="alert alert-block alert-info">
    
**Task:** Check if there are cases of several different `ProductID`s for the same `TransactionID`.
    
</div>

In [ ]:
# Check the TransactionHistory DataFrame for cases of several different ProductIDs for the same TransactionID

In [ ]:
# Check the TransactionHistory DataFrame for cases of several different ProductIDs for the same TransactionID
# First group the dataframe by TransactionID and aggregate the other columns by counting different values
transaction_history_df_grouped = transaction_history_df.groupby(
    ["TransactionID"]
).count()

# Then check if there are results there cells inb ProductID there the count of different values is greater than one
transaction_history_df_grouped[transaction_history_df_grouped["ProductID"] > 1]

# No results => There are no cases of several different ProductIDs for the same TransactionID

Since there are obviously no cases of multiple different `ProductID`s for the same `TransactionID`, our first hypothesis does not seem to be correct. Apparently, the `TransactionID` is the primary key for the `TransactionHistory` relation: i.e. one and the same `TransactionID` cannot refer to different `ProductID`s. 

However, if you look at the `TransactionHistory` again, a second attribute stands out. The `ReferenceOrderID`. This could identify the individual order and products that are part of the same order were obviously purchased together.

So let's test this new hypothesis as well.

<div class="alert alert-block alert-info">
    
**Task:** Check if there are cases of several different `ProductID`s for the same `ReferenceOrderID`.
    
</div>

In [ ]:
# Check the TransactionHistory DataFrame for cases of several different ProductIDs for the same ReferenceOrderID

In [ ]:
# Check the TransactionHistory DataFrame for cases of several different ProductIDs for the same ReferenceOrderID
# Now group the dataframe by ReferenceOrderID and aggregate the other columns by counting different values
transaction_history_df_grouped = transaction_history_df.groupby(
    ["ReferenceOrderID"]
).count()

# Then check again if there are results there cells inb ProductID there the count of different values is greater than one
transaction_history_df_grouped[transaction_history_df_grouped["ProductID"] > 1]

# 23249 results => There are multiple cases of different ProductIDs for the same ReferenceOrderID

Our new hypothesis seems to be correct. I.e. in the next step we want to search for `ProductID`s that regularly occur in the same `ReferenceOrderID`: The frequent itemsets of our problem.

##### Identifing the Frequent Itemsets

To be able to determine our frequent itemsets using mlxtend, we first need to do some preprocessing on `TransactionHistory`.

<div class="alert alert-block alert-info">
    
**Task:** Aggregate the `TransactionHistory` so that next to each `ReferenceOrderID`, the associated `ProductID`s are listed in a single cell.
    
</div>

In [ ]:
# Aggregate the TransactionHistory to have a list of ProductIDs per ReferenceOrderID

In [ ]:
# Aggregate the TransactionHistory to have a list of ProductIDs per ReferenceOrderID
products_per_order_df = (
    transaction_history_df.groupby("ReferenceOrderID")["ProductID"]
    .apply(list)
    .reset_index(name="ProductIDs")
    .set_index("ReferenceOrderID")
)
products_per_order_df

<div class="alert alert-block alert-info">
    
**Task:** Prepare the dataset for `mlxtend`s `fpgrowth` by using the `TransactionEncoder` of the library
    
</div>

In [ ]:
# Apply one hot encoding to the prepared dataset by using the TransactionEncoder

In [ ]:
# Apply one hot encoding to the prepared dataset by using the TransactionEncoder
# Create a TransactionEncoder
transaction_encoder = TransactionEncoder()

# Use the TransactionEncoder to transform the dataset into a one-hot encoded NumPy boolean array
one_hot_encoded_dataset = transaction_encoder.fit(
    products_per_order_df["ProductIDs"].tolist()
).transform(products_per_order_df["ProductIDs"].tolist())

# Transform the one-hot encoded array into a pandas DataFrame
preprocessed_dataset = pd.DataFrame(
    one_hot_encoded_dataset,
    columns=transaction_encoder.columns_,
    index=products_per_order_df.index,
)
preprocessed_dataset

After these necessary preprocessing steps, the frequent itemsets can now theoratically be determined. However, we definitely do not know at this point which min_support to choose. 

Even by trial and error, it is difficult to find a meaningful threshold here, since we have only been told by our fictitious bosses that we should find the ten most "relevant" frequent patterns.

First of all, it would therefore make sense to determine rather too many itemsets than too few. It is easier to discard frequent itemsets later than to create additional ones.

<div class="alert alert-block alert-info">
    
**Task:** Use `fpgrowth` to determine the frequent itemsets of our dataset. Select `min_support` so that the approximately 100 most frequent itemsets become frequent itemsets.
    
</div>

In [ ]:
# Determine the frequent itemsets

In [ ]:
# Determine the frequent itemsets
frequent_itemsets = fpgrowth(preprocessed_dataset, min_support=0.01, use_colnames=True)
frequent_itemsets

##### Determination of the Frequent Patterns

Before we use the frequent itemsets to determine frequent patterns, we must first determine how to define "relevance" in the context of frequent patterns.

If our bosses meant that they wanted to know which ten patterns occur most frequently in our dataset, then support would be the appropriate measure. Did they want to know how certain one can be that Product A will end up in the shopping cart if Product B is already there? Then the calculation of confidence would be more appropriate. In addition, there are of course a large number of interestingness measures.

All in all, this question cannot be answered conclusively. In practice, a dialog between management and you would be appropriate in order to narrow down more precisely what is meant by the most "relevant" ten patterns.  

This ambiguity was intentionally used in the assignment to show that the assignment will often contain inaccuracies in the real world. 

However, while in the real world dialogue is the best solution, we have no opportunity to consult with our fictitious bosses. For this reason we do what is best for us and choose the simplest measure to apply: The support.

It is not important at first that they really generate only the 10 rules with the highest support. If they are part of your list, everything is fine.

<div class="alert alert-block alert-info">
    
**Task:** Use `mlxtend`s `association_rules` to generate frequent patterns from the frequent itemsets. Set the corresponding threshold so that at least the 10 frequent patterns with the highest support are included.
    
</div>

In [ ]:
# Generate the association rules/frequent patterns

In [ ]:
# Generate the association rules/frequent patterns
frequent_patterns = association_rules(
    frequent_itemsets, metric="support", min_threshold=0.02
)
frequent_patterns

Of course, it is no problem at all to sort out extra patterns afterwards. 

<div class="alert alert-block alert-info">
    
**Task:** Delete all patterns that do not belong to the ten patterns with the highest support.
    
</div>

In [ ]:
# Delete the extra patterns

In [ ]:
# Delete the extra patterns
frequent_patterns = frequent_patterns.nlargest(10, "support")
frequent_patterns

##### Getting to Know the Product Names 

Even though we have now already completed the core task, it will probably do management little good to tell them that `ProductID` 871 is often purchased in addition to `ProductID` 870. After all, these are first and foremost internal database ids. 

To complete our task satisfactorily for the management, we still need to enrich the `ProductID`s in antecedents and consequents with their actual names.

<div class="alert alert-block alert-info">
    
**Task:** Enrich the frequent patterns by adding the product names to the list. 
    
</div>

In [ ]:
# Merge the ProductName into the frequent pattern df

In [ ]:
# Merge the ProductName into the frequent pattern df
# We have to transform the frozensets within the two colums to strings first
# (as we know that there is only one item per set this is pretty simple)
frequent_patterns["antecedents"] = frequent_patterns["antecedents"].apply(
    lambda x: list(x)[0]
)
frequent_patterns["consequents"] = frequent_patterns["consequents"].apply(
    lambda x: list(x)[0]
)

# After that we have to merge frequent_patterns with the product df
frequent_patterns = pd.merge(
    frequent_patterns, product_df, left_on="antecedents", right_on="ProductID"
)[
    [
        "antecedents",
        "Name",
        "consequents",
        "antecedent support",
        "consequent support",
        "support",
        "confidence",
        "lift",
        "leverage",
        "conviction",
    ]
]
frequent_patterns = frequent_patterns.rename(columns={"Name": "antecedents name"})
frequent_patterns = pd.merge(
    frequent_patterns, product_df, left_on="consequents", right_on="ProductID"
)[
    [
        "antecedents",
        "antecedents name",
        "consequents",
        "Name",
        "antecedent support",
        "consequent support",
        "support",
        "confidence",
        "lift",
        "leverage",
        "conviction",
    ]
]
frequent_patterns = frequent_patterns.rename(columns={"Name": "consequents name"})

# Print the df
frequent_patterns

Assignment completed! In the virtual scenario introduced in the beginning of this part you would now be able to report to the management that the `Mountain Bottle Cage` is often purchased in addition to the `Water Bottle - 30 oz.`. The same is true for the other nine requested frequent patterns.


### Implementing the Kulczynski Measure and the Imbalance Ratio

The library `mlxtend` offers some more measures besides support and confidence for the determination of frequent patterns. While lift, leverage and conviction are offered, the kulczynski metric and imbalance ratio presented in the lecture, for example, are not.

Fortunately, the the antecedent support, the consequent support and the support calculated by `mlxtend` can easily be used to calculate these two interestingness measures.

<div class="alert alert-block alert-info">
    
**Task:** Write a function to compute the kulczynski measure, known from the lecture as `Kulc(a, b)`.
    
</div>

In [ ]:
# Complete the function kulczynski_measure to compute the kulczynski measure
def kulczynski_measure(antecedent_support, consequent_support, support):
    # ...
    return 0


# Compute the kulczynski measure for "Water Bottle - 30 oz." -> "Mountain Bottle Cage"
kulczynski_measure(0.112802, 0.049356, 0.041139)

In [ ]:
# Complete the function kulczynski_measure to compute the kulczynski measure
def kulczynski_measure(antecedent_support, consequent_support, support):
    # Simply use the formula introduced in the lecture
    return (support / 2) * ((1 / antecedent_support) + (1 / consequent_support))


# Compute the kulczynski measure for "Water Bottle - 30 oz." -> "Mountain Bottle Cage"
kulczynski_measure(0.112802, 0.049356, 0.041139)

<div class="alert alert-block alert-info">
    
**Task:** Write a function to compute the imbalance ratio.
    
</div>

In [ ]:
# Complete the function imbalance_ratio to compute the imbalance ratio
def imbalance_ratio(antecedent_support, consequent_support, support):
    # ...
    return 0


# Compute the imbalance ratio for "Water Bottle - 30 oz." -> "Mountain Bottle Cage"
imbalance_ratio(0.112802, 0.049356, 0.041139)

In [ ]:
# Complete the function imbalance_ratio to compute the imbalance ratio
def imbalance_ratio(antecedent_support, consequent_support, support):
    # Simply use the formula introduced in the lecture
    return abs(antecedent_support - consequent_support) / (
        antecedent_support + consequent_support - support
    )


# Compute the imbalance ratio for "Water Bottle - 30 oz." -> "Mountain Bottle Cage"
imbalance_ratio(0.112802, 0.049356, 0.041139)

Of course, the question arises again as to how these metrics are to be interpreted.

<div class="alert alert-block alert-info">
    
**Task:** Interpret the interestingness measures for the association rule `"Water Bottle - 30 oz." -> "Mountain Bottle Cage"`
    
</div>

Write down your solution here:

First both values must be interpreted separately from each other:

- **Kulczynski Measure:**<br/>
When kulczynski measure is close to 0 or 1 we have an "interesting" association rule. Since in this case the value is about 0.6, the kulczynski measure rather suggests that this association rule is uninteresting. 
- **Imbalance Ratio:**<br/>
For the imbalance ratio, a value of 0 indicates a perfectly balanced association rule, while 1 indicates a very unbalanced one. In this case, we are about 0.52, which is about the middle of the spectrum. Thus, we cannot speak of a particularly well balanced rule, but neither can we speak of a completely unbalanced one.

In summary, we have not discovered the most interesting rule, but one that is not completely uninteresting (this would the case for kulczynski measure = 0.5 and imbalance ratio = 0.0). 